In [30]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.autograd import Variable
dtype = torch.FloatTensor

In [31]:
sentences = ['i like dog','i love coffee','i hate milk','i like nlp']

word_list = ' '.join(sentences).split()
word_list = list(set(word_list))    
# print(word_list)

# create dict
word_dict = {w:i for i,w in enumerate(word_list)}   
number_dict = {i:w for i ,w in enumerate(word_list)}
# print(word_dict)
# print(number_dict)

In [32]:
n_num = len(word_dict)                           #dict length

In [61]:
m = 2                                            #Vector dimension of words

def make_batch(sentence):
    input_batch = []
    target_batch = []
    
    for sen in sentence:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]  #word->id
        target = word_dict[word[-1]]
        input_batch.append(input)
        target_batch.append(target)
    return input_batch,target_batch

n_step = 2       #2gram
n_hidden = 2

class NNLM(nn.Module):
    def __init__(self):
        super(NNLM, self).__init__()
        self.embed = nn.Embedding(n_num, m)     #input id ,output vector
#         parameter 
        self.W = nn.Parameter(torch.randn(n_step * m, n_hidden).type(dtype))
        self.d = nn.Parameter(torch.randn(n_hidden).type(dtype))
        self.U = nn.Parameter(torch.randn(n_hidden, n_num).type(dtype))
        self.b = nn.Parameter(torch.randn(n_num).type(dtype))
            
    def forward(self, x):
        x = self.embed(x)  # 4 * 2 * 2
        x = x.view(-1, n_step * m)
        tanh = torch.tanh(torch.mm(x, self.W) + self.d)  #4 * 2 
        output = self.b + torch.mm(tanh, self.U)
        return output

model = NNLM()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)  #optimizer

# traning
input_batch, target_batch = make_batch(sentences)
input_batch = Variable(torch.LongTensor(input_batch))
target_batch = Variable(torch.LongTensor(target_batch))

for epoch in range(5000):
    optimizer.zero_grad()
    
    output = model(input_batch)   # input 4 * 2
    
    loss = criterion(output, target_batch)
    
    if (epoch + 1) % 1000 == 0:
        print('epoch:','%04d' %(epoch + 1), 'cost = {:.6f}'.format(loss.item()))
    
    loss.backward()
    optimizer.step()
    
# predict
predict = model(input_batch).data.max(1, keepdim = True)[1]

print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

epoch: 1000 cost = 1.363340
epoch: 2000 cost = 0.581180
epoch: 3000 cost = 0.413003
epoch: 4000 cost = 0.376426
epoch: 5000 cost = 0.362538
[['i', 'like'], ['i', 'love'], ['i', 'hate'], ['i', 'like']] -> ['nlp', 'coffee', 'milk', 'nlp']
